In [ ]:
import pandas as pd
import numpy as np
from pandas import Series

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_df = pd.read_csv('/kaggle/input/layer7/train.csv')
valid_df = pd.read_csv('/kaggle/input/layer7/valid.csv')
test_df = pd.read_csv('/kaggle/input/layer7/test.csv')

In [ ]:
train_df.shape

In [ ]:
train_df.head()

In [ ]:
missing_columns = train_df.columns[train_df.isnull().any()]
missing_counts = train_df[missing_columns].isnull().sum()

print('Missing Columns and Counts')
for column in missing_columns:
    print( str(column) +' : '+ str(missing_counts[column]))

In [ ]:
train_data = train_df.copy()
valid_data = valid_df.copy()
test_data = test_df.copy()

In [ ]:
train_df.describe()

In [ ]:
from sklearn.preprocessing import RobustScaler # eliminate outliers

x_train = {}
x_valid = {}
x_test = {}

y_train = {}
y_valid = {}
y_test = {}

#create dictionaries for each label
for target_label in ['label_1','label_2','label_3','label_4']:

  if target_label == "label_2":
    train = train_df[train_df['label_2'].notna()]
    valid = valid_df[valid_df['label_2'].notna()]
  else:
    train = train_df
    valid = valid_df

  test = test_df

  scaler = RobustScaler()

  x_train[target_label] = pd.DataFrame(scaler.fit_transform(train.drop(['label_1','label_2','label_3','label_4'], axis=1)), columns=[f'feature_{i}' for i in range(1,769)])
  y_train[target_label] = train[target_label]

  x_valid[target_label] = pd.DataFrame(scaler.transform(valid.drop(['label_1','label_2','label_3','label_4'], axis=1)), columns=[f'feature_{i}' for i in range(1,769)])
  y_valid  [target_label] = valid[target_label]

  x_test[target_label] = pd.DataFrame(scaler.transform(test.drop(["ID"],axis=1)), columns=[f'feature_{i}' for i in range(1,769)])

In [ ]:
x_train_df = x_train['label_4'].copy()
y_train_df = y_train['label_4'].copy()

x_valid_df = x_valid['label_4'].copy()
y_valid_df = y_valid['label_4'].copy()

x_test_df = x_test['label_4'].copy()

# Cross Validation

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, KFold

# Perform cross-validation
scores = cross_val_score(SVC(), x_train_df, y_train_df, cv=5, scoring='accuracy')

mean_accuracy = scores.mean()
std_accuracy = scores.std()
# Print the cross-validation scores
print('Support Vector Machines')
print('\n')
print("Cross-validation scores:", scores)
print(f"Mean Accuracy: {mean_accuracy:.2f}")
print(f"Standard Deviation: {std_accuracy:.2f}")

# Feature Engineering

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.975, svd_solver='full')
pca.fit(x_train_df)
x_train_df_pca = pd.DataFrame(pca.transform(x_train_df))
x_valid_df_pca = pd.DataFrame(pca.transform(x_valid_df))
x_test_df_pca = pd.DataFrame(pca.transform(x_test_df))
print('Shape after PCA: ',x_train_df_pca.shape)

In [ ]:
from sklearn import metrics

# SVM

In [ ]:
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

classifier = svm.SVC(kernel='linear', C=1)

classifier.fit(x_train_df_pca, y_train_df)

y_valid_pred = classifier.predict(x_valid_df_pca)

print("acc_score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

In [ ]:
# from sklearn.svm import SVC
# from sklearn.model_selection import RandomizedSearchCV
# from scipy.stats import uniform, randint
# import numpy as np

# param_dist = {
#     'C': [100,10,1,0,0.1,0.01],
#     'kernel': ['rbf','linear','poly','sigmoid'],
#     'gamma': ['scale','auto'],
#     'degree': [1,2,3,4],
#     'class_weight' : ['none','balanced']
# }

# svm = SVC()

# random_search = RandomizedSearchCV(
#     svm, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1, random_state=42, scoring='accuracy'
# )

# random_search.fit(x_train_df_pca, y_train_df)

# best_params = random_search.best_params_
# best_model = random_search.best_estimator_

# print("best parameters:", best_params)

Label 1 
best parameters: {'kernel': 'rbf', 'gamma': 'scale', 'degree': 4, 'class_weight': 'balanced', 'C': 100}

In [ ]:
from sklearn import svm

classifier = svm.SVC(kernel='rbf', C=100, gamma='scale', degree=4, class_weight='balanced')

classifier.fit(x_train_df_pca, y_train_df)

y_valid_pred = classifier.predict(x_valid_df_pca)

print("acc_score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

y_test_predict_after_pca = classifier.predict(x_test_df_pca)

# RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=100, random_state=42)

classifier.fit(x_train_df, y_train_df)

y_valid_pred = classifier.predict(x_valid_df)

print("accuracy_score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

y_test_pred = classifier.predict(x_test_df)

# CSV Creation

In [ ]:
output_df=pd.DataFrame(columns=["ID","label_1","label_2","label_3","label_4"])

In [ ]:
IDs = list(i for i in range(1, len(test_df)+1))
output_df["ID"] = IDs

In [ ]:
output_df["label_4"] = y_test_predict_after_pca

In [ ]:
output_df

In [ ]:
output_df.to_csv('/kaggle/working/output7_4.csv',index=False)